In [1]:
import numpy as np
import pandas as pd
import os

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.utils import to_categorical

# Biosppy for signal processing
from biosppy.signals import ecg

# Randomly seeding
tf.random.set_seed(6950)

# Constants
SAMPLING_RATE = 500  # Hz
ecg_folder = "../../../Datasets/12-lead electrocardiogram database/ECGData"
diagnostics_file = "../../../Datasets/12-lead electrocardiogram database/Diagnostics.xlsx"

# Label mapping
rhythm_mapping = {
    'AFIB': 'AFIB',
    'AF': 'AFIB',
    'SVT': 'GSVT',
    'AT': 'GSVT',
    'SAAWR': 'GSVT',
    'ST': 'GSVT',
    'AVNRT': 'GSVT',
    'AVRT': 'GSVT',
    'SB': 'SB',
    'SR': 'SR',
    'SA': 'SR'
}

2024-11-29 08:37:55.328613: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-29 08:37:55.339084: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-29 08:37:55.342288: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-29 08:37:55.351081: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-29 08:37:55.854935: W tensorflow/compiler/tf2

In [2]:
def extract_comprehensive_features(ecg_signal):
    """
    Extract comprehensive features from entire ECG signal
    """
    # Process ECG signal
    try:
        ecg_processed = ecg.ecg(signal=ecg_signal, sampling_rate=SAMPLING_RATE, show=False)

        # R-peak related features
        rpeaks = ecg_processed['rpeaks']
        heart_rate = ecg_processed['heart_rate']

        # Time domain features
        features = [
            # Statistical features of raw signal
            np.mean(ecg_signal),
            np.std(ecg_signal),
            np.median(ecg_signal),
            np.min(ecg_signal),
            np.max(ecg_signal),

            # R-peak related features
            len(rpeaks),  # Number of R peaks
            np.mean(heart_rate) if len(heart_rate) > 0 else 0,
            np.std(heart_rate) if len(heart_rate) > 0 else 0,

            # R-R interval features
            np.mean(np.diff(rpeaks)) if len(rpeaks) > 1 else 0,
            np.std(np.diff(rpeaks)) if len(rpeaks) > 1 else 0,

            # Additional signal characteristics
            np.percentile(ecg_signal, 25),
            np.percentile(ecg_signal, 75),
        ]

        return features

    except Exception as e:
        print(f"Error processing signal: {e}")
        return [0] * 12


def prepare_dataset(ecg_folder, diagnostics_df):
    """
    Prepare dataset by processing whole ECG signals
    """
    signals = []
    signal_features = []
    signal_labels = []

    for _, row in diagnostics_df.iterrows():
        file_name = row['FileName']
        rhythm_label = row['Rhythm']

        # Skip if rhythm label is invalid
        if pd.isnull(rhythm_label) or rhythm_label not in rhythm_mapping.values():
            continue

        # Load ECG file (Lead 2)
        ecg_file = os.path.join(ecg_folder, f"{file_name}.csv")
        if not os.path.exists(ecg_file):
            continue

        # Load entire signal
        ecg_data = pd.read_csv(ecg_file, header=0).iloc[:, 1].values
        ecg_data = ecg_data.astype(float)

        # Extract features from whole signal
        features = extract_comprehensive_features(ecg_data)

        signals.append(ecg_data)
        signal_features.append(features)
        signal_labels.append(rhythm_label)

    return (np.array(signals),
            np.array(signal_features),
            np.array(signal_labels))

In [3]:
# Load diagnostics
diagnostics_df = pd.read_excel(diagnostics_file)
diagnostics_df['Rhythm'] = diagnostics_df['Rhythm'].map(rhythm_mapping)

# Prepare dataset
raw_signals, signal_features, signal_labels = prepare_dataset(ecg_folder, diagnostics_df)
print(raw_signals.shape, signal_features.shape, signal_labels.shape)

(10646, 5000) (10646, 12) (10646,)


In [4]:
# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(signal_labels)
onehot_labels = to_categorical(encoded_labels)

# Scale features
feature_scaler = StandardScaler()
scaled_features = feature_scaler.fit_transform(signal_features)

# Pad signals for neural network input
max_length = max(len(signal) for signal in raw_signals)
padded_signals = np.array([np.pad(signal, (0, max_length - len(signal))) for signal in raw_signals])


# Create model with multiple architectures
def create_hybrid_model(input_shape, num_classes):
    model = Sequential([
        # 1D Convolutional layers
        Conv1D(64, 3, activation='relu', input_shape=input_shape),
        MaxPooling1D(2),
        Conv1D(32, 3, activation='relu'),
        MaxPooling1D(2),

        # LSTM layer
        LSTM(50, return_sequences=False),

        # Dense layers with features
        Dense(64, activation='relu', input_shape=(scaled_features.shape[1],)),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.2),

        # Output layer
        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


# Stratified K-Fold Cross-Validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []

for train_index, test_index in skf.split(padded_signals, signal_labels):
    # Split data
    X_train_signals = padded_signals[train_index]
    X_train_features = scaled_features[train_index]
    y_train = onehot_labels[train_index]

    X_test_signals = padded_signals[test_index]
    X_test_features = scaled_features[test_index]
    y_test = onehot_labels[test_index]

    # Create and train model
    model = create_hybrid_model(
        (X_train_signals.shape[1], 1),
        onehot_labels.shape[1]
    )

    # Training
    history = model.fit(
        [X_train_signals.reshape(-1, X_train_signals.shape[1], 1), X_train_features],
        y_train,
        epochs=50,
        batch_size=32,
        validation_split=0.2,
        verbose=1
    )

    # Evaluate
    y_pred = model.predict([
        X_test_signals.reshape(-1, X_test_signals.shape[1], 1),
        X_test_features
    ])
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)

    # Compute accuracy
    accuracy = np.mean(y_pred_classes == y_test_classes)
    cv_scores.append(accuracy)

    print("\nClassification Report:")
    print(classification_report(
        y_test_classes,
        y_pred_classes,
        target_names=label_encoder.classes_
    ))

# Print cross-validation results
print("\nCross-Validation Scores:", cv_scores)
print("Mean CV Accuracy:", np.mean(cv_scores))

/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1732848168.152546  504286 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732848168.182687  504286 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:

Epoch 1/50


2024-11-29 08:42:49.752489: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
W0000 00:00:1732848169.862713  507256 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732848169.894763  507256 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732848169.895411  507256 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732848169.896048  507256 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732848169.906182  507256 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732848169.908448  507256 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732848169.910930  507256 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732848169.918394  507256 gpu_t

  1/213 ━━━━━━━━━━━━━━━━━━━━ 8:38 2s/step - accuracy: 0.3125 - loss: 1.4186

W0000 00:00:1732848170.972381  507255 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732848170.973361  507255 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732848170.974523  507255 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732848170.975563  507255 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732848170.976571  507255 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732848170.980907  507255 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732848170.982655  507255 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732848170.983806  507255 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732848170.984870  507255 gp

...

Epoch 1/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - accuracy: 0.3611 - loss: 1.3423 - val_accuracy: 0.3146 - val_loss: 1.4360
Epoch 2/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.3899 - loss: 1.2834 - val_accuracy: 0.3245 - val_loss: 1.4230
Epoch 3/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 8s 35ms/step - accuracy: 0.4039 - loss: 1.2705 - val_accuracy: 0.4777 - val_loss: 1.3252
Epoch 4/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.4510 - loss: 1.1954 - val_accuracy: 0.6344 - val_loss: 1.0494
Epoch 5/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.5159 - loss: 1.1068 - val_accuracy: 0.4542 - val_loss: 1.3456
Epoch 6/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.4630 - loss: 1.2032 - val_accuracy: 0.5880 - val_loss: 1.0977
Epoch 7/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.5202 - loss: 1.0941 - val_accuracy: 0.6209 - val_loss: 1.0360
Epoch 8/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.5694 - loss: 0.9854 - val_accu

/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.3852 - loss: 1.3232 - val_accuracy: 0.2330 - val_loss: 1.4645
Epoch 2/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.3920 - loss: 1.2921 - val_accuracy: 0.2518 - val_loss: 1.4107
Epoch 3/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.4003 - loss: 1.2726 - val_accuracy: 0.3316 - val_loss: 1.3613
Epoch 4/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.4313 - loss: 1.2346 - val_accuracy: 0.5411 - val_loss: 1.2534
Epoch 5/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 8s 35ms/step - accuracy: 0.4934 - loss: 1.1375 - val_accuracy: 0.6015 - val_loss: 1.0958
Epoch 6/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.5554 - loss: 1.0309 - val_accuracy: 0.5921 - val_loss: 1.0610
Epoch 7/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.6066 - loss: 0.9264 - val_accuracy: 0.5968 - val_loss: 1.0806
Epoch 8/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.6202 - loss: 0.8826 - val_accu

/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - accuracy: 0.3628 - loss: 1.3565 - val_accuracy: 0.2523 - val_loss: 1.4627
Epoch 2/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.3976 - loss: 1.2895 - val_accuracy: 0.2488 - val_loss: 1.4935
Epoch 3/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.4172 - loss: 1.2618 - val_accuracy: 0.5246 - val_loss: 1.2702
Epoch 4/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.4781 - loss: 1.1627 - val_accuracy: 0.4836 - val_loss: 1.3177
Epoch 5/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.5415 - loss: 1.0640 - val_accuracy: 0.5669 - val_loss: 1.1122
Epoch 6/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.6243 - loss: 0.9001 - val_accuracy: 0.5775 - val_loss: 1.0861
Epoch 7/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.6348 - loss: 0.8772 - val_accuracy: 0.5710 - val_loss: 1.0974
Epoch 8/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 8s 37ms/step - accuracy: 0.6639 - loss: 0.8198 - val_accu